In [1]:
# importing required modules
import pandas as pd
import mysql.connector as msql
import math
import os
import datetime

In [2]:
#display all columns and rows
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [3]:
#Reading She for STEM Incubator file present on source files
directory_path =(r"C:\Users\HP\OneDrive - VigyanShaala\02 Products  Initiatives\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Remapping\Kalpana Source File\02_Source_Kalpana incubator WN")
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

for file in csv_files:
    file_path = os.path.join(directory_path, file)
    da = pd.read_csv(file_path)
    print(f"Data from {file}:")

Data from Kalpana Incubator Sept- Dec 2023 Batch-1704361632176.csv:


In [4]:
# Create a new dataframe with 'email' column and columns that start with 'Week' 'Video', 'Recording', and 'Master class'
da = da[['Email'] + [col for col in da.columns if col.startswith(('Week','Video', 'SUK', 'Master Class'))]]


In [5]:
oldcol = list(da.columns)


In [6]:
oldcol = [col for col in oldcol if not col.startswith("Week")]
oldcol = [col.split(" :")[0] for col in oldcol]
oldcol = [col.split(": ")[0] for col in oldcol]

In [7]:
def rename_column(col):
    # Split the column name by space
    parts = col.split()
    special_chars = {'!': '1', '@': '2', '#': '3', '$': '4', '%': '5', '^': '6', '&': '7', '*': '8', '?': '9', '+': '0'}
    
    # If the first part is 'SUK', add the number corresponding to the last character
    if parts[0] == 'SUK':
        return parts[0] + special_chars.get(parts[1][-1], '0')
    
    # If the first part is 'Video', use the prefix 'VID' and the numbers corresponding to the last two characters
    elif parts[0] == 'Video':
        last_char = parts[1][-1]
        second_last_char = parts[1][-2] if len(parts[1]) > 1 else '+'
        return 'VID' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')
    
    # If the first part contains 'Master' and 'Class' is present, use the prefix 'MC' and the numbers corresponding to the last two characters
    elif 'Master' in parts and 'Class' in parts:
        index = parts.index('Master')
        last_char = parts[index + 2][-1]
        second_last_char = parts[index + 2][-2] if len(parts[index + 2]) > 1 else '+'
        return 'MC' + special_chars.get(second_last_char, '0') + special_chars.get(last_char, '0')
    
    # Otherwise, return the column name as it is
    else:
        return col

# Apply the rename_column function to each element in the oldcol list
oldcol = [rename_column(col) for col in oldcol]




In [8]:
# Reading Incubator_and_attendence_monitoring from Outout files
data =pd.read_csv(r"C:\Users\HP\OneDrive - VigyanShaala\02 Products  Initiatives\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Remapping\Kalpana Output\Incubator_and_attendence_monitoring_Sep_2023.csv")

In [9]:
data.head(25)

,Email,WK0_SUK_V,WK0_Master1,WK1_V1,WK1_V2,WK1_Master2,WK2_V1,WK2_V2,WK2_V3,WK2_V4,WK2_Master3,WK3_V1,WK3_V2,WK3_V3,WK3_V4,WK3_Master4,WK4_V1,WK4_V2,WK4_V3,WK4_V4,WK4_V5,WK4_V6,WK4_V7,WK4_Master5,WK5_V1,WK6_SUK_V,WK7_V1,WK8_SUK_1,WK8_SUK_2,Recorded_Total,Recorded_Average,Recorded_Percentage,SUK_Recorded_Total,SUK_Recorded_Average,SUK_Recorded_Percentage,Masterclass_Total,Masterclass_Average,Masterclass_Percentage,Program_Total,Program_Average,Program_percentage
0,vijayacads@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.25,0.04,0.01,0.89,0.05,0.00,0.18
1,sreejith.sreenivasan@vigyanshaala.com,0.63,0.47,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.09,0.63,0.32,31.54,0.50,0.10,9.95,1.13,0.04,4.09
2,monikadaw5353@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,prachijagdale0123@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,nikitasingh4615@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.44,0.00,0.0,0.00,0.78,0.0,0.00,0.00,0.00,0.00,0.00,0.00,1.44,0.29,20.02,2.22,0.08,6.35
5,keerthishivampeta51@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,habibrutuja2003@gmail.com,0.00,0.13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.13,0.03,2.61,0.13,0.00,0.47
7,florenceadekojo59@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,sreevishnumohanraj@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.01
9,bhargavigpatil64@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.16,0.00,0.00,0.06,0.00,0.00,0.02,0.00,0.00,0.12


In [10]:
df=pd.DataFrame(data["Email"])

In [11]:
# Get the list of columns starting with 'WK' or 'Email'
selected_columns = [col for col in data.columns if col.startswith('WK') or col.lower() == 'email']


In [12]:
# Keep only the selected columns in the DataFrame
data = data[selected_columns]

In [13]:
rename_col=list(data.columns)

In [14]:
data.columns = oldcol 

In [15]:
data.head(25)

,Email,SUK0,MC01,VID01,VID02,MC02,VID03,VID04,VID05,VID06,MC03,VID07,VID08,VID09,VID10,MC04,VID11,VID12,VID13,VID14,VID15,VID16,VID17,MC05,VID18,SUK1,VID20,SUK2,SUK3
0,vijayacads@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.0,0.00,0.00,0.0
1,sreejith.sreenivasan@vigyanshaala.com,0.63,0.47,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
2,monikadaw5353@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
3,prachijagdale0123@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
4,nikitasingh4615@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.44,0.00,0.0,0.00,0.78,0.0
5,keerthishivampeta51@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
6,habibrutuja2003@gmail.com,0.00,0.13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
7,florenceadekojo59@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
8,sreevishnumohanraj@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
9,bhargavigpatil64@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0


In [16]:
# Reading Excel file form our source
excel_file  = pd.read_excel(r"C:\Users\HP\OneDrive - VigyanShaala\02 Products  Initiatives\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Remapping\Kalpana Source File\00_Source_Kalpana Video Details\Timestamp_of_Videos.xlsx")

In [17]:
excel_file.head()

,Content Name,Code,Time,Notation of Code,Number,Notation
0,"SUK + : Kalpana Inauguration, at 6:30 PM. on S...",SUK0,3600,+,0.0,+
1,Master Class ! : Career Exploration - Saturday...,MC01,3600,!,1.0,!
2,Video ! : Why women in STEM. What has Kalpana ...,VID01,291,!,2.0,@
3,Video @ : What we expect from our Kalpana fellows,VID02,287,@,3.0,#
4,Master Class @ : Planning your Applications – ...,MC02,3600,@,4.0,$


In [18]:
# Iterate over each column in the 'data' DataFrame
for column_name in data.columns:
    # Check if the column exists in the Excel file
    if column_name in excel_file['Code'].values:
        # Get the corresponding value in the "Time" column from the Excel file
        value = excel_file.loc[excel_file['Code'] == column_name, 'Time'].values[0]
        
        # Calculate the percentage value
        percentage_value = (data[column_name] * 100) / (value/3600)
        
        # Assign the calculated percentage value to a new column in the "data1" DataFrame with the same name as the "Code" column
        df[column_name] = percentage_value
    else:
        # Handle the case when the column is present in 'data' but not in the Excel file
        print(f"Column '{column_name}' present in 'data' but not in the Excel file. Skipping.")


Column 'Email' present in 'data' but not in the Excel file. Skipping.


In [19]:
data.head(25)

,Email,SUK0,MC01,VID01,VID02,MC02,VID03,VID04,VID05,VID06,MC03,VID07,VID08,VID09,VID10,MC04,VID11,VID12,VID13,VID14,VID15,VID16,VID17,MC05,VID18,SUK1,VID20,SUK2,SUK3
0,vijayacads@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.0,0.00,0.00,0.0
1,sreejith.sreenivasan@vigyanshaala.com,0.63,0.47,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
2,monikadaw5353@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
3,prachijagdale0123@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
4,nikitasingh4615@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.44,0.00,0.0,0.00,0.78,0.0
5,keerthishivampeta51@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
6,habibrutuja2003@gmail.com,0.00,0.13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
7,florenceadekojo59@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
8,sreevishnumohanraj@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
9,bhargavigpatil64@gmail.com,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0


In [20]:
# Renaming column name with old name
df.columns = rename_col 
data.columns = rename_col 

In [21]:
# Changing percent values greater than 100 to 100
def Max_Value(value):
    if value >=100:
        return 100
    else:
        return value

In [22]:
col=df.iloc[:,1:]
df1=col.columns
df[df1]=df[df1].applymap(Max_Value)

# Define Week Here  ↓

In [23]:
end_week = 8

# Weekly Recorded Total Hours

In [24]:
cols_to_select = ['Email']  # Start with 'email' column as the first column
for week in range(end_week + 1):
    video_cols = [f'WK{week}_V{video}' for video in range(1, 10)]
    video_cols = [col for col in video_cols if col in data.columns and col != 'email']
    cols_to_select.extend(video_cols)

# Select the desired columns and create a new DataFrame
Weekly = data[cols_to_select].copy()

# Calculate total for each week and add to Weekly dataframe
for week in range(end_week + 1):
    video_cols = [f'WK{week}_V{video}' for video in range(1, 10)]
    video_cols = [col for col in video_cols if col in Weekly.columns]
    Weekly[f'WK{week}_Recorded_Total'] = Weekly[video_cols].sum(axis=1, skipna=True)

# Calculate total for all weeks and add to Weekly dataframe
total_cols = [f'WK{week}_Recorded_Total' for week in range(end_week + 1) if f'WK{week}_Recorded_Total' in Weekly.columns]
Weekly['All_Week_Total'] = Weekly[total_cols].sum(axis=1, skipna=True)

# Select the desired columns in the Weekly dataframe
cols_to_select = ['Email'] + total_cols + ['All_Week_Total']
Weekly = Weekly[cols_to_select]


# Weekly average of Recorded Videos

cols_to_select = ['Email']  # Start with 'email' column as the first column
for week in range(end_week + 1):
    video_cols = [f'WK{week}_V{video}' for video in range(1, 10)]
    video_cols = [col for col in video_cols if col in data.columns and col != 'email']
    cols_to_select.extend(video_cols)

# Select the desired columns and create a new DataFrame
Weekly_AVG = data[cols_to_select].copy()

# Calculate total for each week and add to Weekly dataframe
for week in range(end_week + 1):
    video_cols = [f'WK{week}_V{video}' for video in range(1, 10)]
    video_cols = [col for col in video_cols if col in Weekly_AVG.columns]
    Weekly_AVG[f'WK{week}_Average'] = Weekly_AVG[video_cols].mean(axis=1, skipna=True)

# Calculate total for all weeks and add to Weekly dataframe
total_cols = [f'WK{week}_Average' for week in range(end_week + 1) if f'WK{week}_Average' in Weekly_AVG.columns]
Weekly_AVG['All_Week_Average'] = Weekly_AVG[total_cols].mean(axis=1, skipna=True)

# Select the desired columns in the Weekly dataframe
cols_to_select = ['Email'] + total_cols + ['All_Week_Average']
Weekly_AVG = Weekly_AVG[cols_to_select]


# Merging Dataframe
Weekly = Weekly.merge(Weekly_AVG,on= 'Email', how ='left')

# Recorded weekly Percentage

In [25]:
cols_to_select = ['Email']  # Start with 'email' column as the first column
for week in range(end_week + 1):
    video_cols = [f'WK{week}_V{video}' for video in range(1, 10)]
    video_cols = [col for col in video_cols if col in df.columns and col != 'email']
    cols_to_select.extend(video_cols)

# Select the desired columns and create a new DataFrame
Weekly_Per = df[cols_to_select].copy()

# Calculate total for each week
for week in range(end_week + 1):
    video_cols = [f'WK{week}_V{video}' for video in range(1, 10)]
    video_cols = [col for col in video_cols if col in Weekly_Per.columns]
    Weekly_Per[f'Recorded_WK{week}_percent'] = Weekly_Per[video_cols].mean(axis=1, skipna=True)

# Calculate total for all weeks
total_cols = [f'Recorded_WK{week}_percent' for week in range(end_week + 1) if f'Recorded_WK{week}_percent' in Weekly_Per.columns]
Weekly_Per['Recorded_All_Week_percent'] = Weekly_Per[total_cols].mean(axis=1, skipna=True)

# Select the desired columns in the Weekly dataframe
cols_to_select = ['Email'] + total_cols + ['Recorded_All_Week_percent']
Weekly_Per = Weekly_Per[cols_to_select]


In [26]:
# Merging Dataframe
Weekly = Weekly.merge(Weekly_Per,on= 'Email', how ='left')

# SUK Weekly Total Hours

In [27]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    col_name = f'WK{week}_SUK_V'
    if col_name in data.columns:
        cols_to_select.append(col_name)

# Add the 'email' column to the list of selected columns
cols_to_select.append('Email')

# Select the desired columns and create a new DataFrame
SUK_Recorded_Total = data.loc[:, cols_to_select]

# Create new columns with 'SUK' prefix for each week
for week in range(end_week + 1):
    col_name = f'WK{week}_SUK_V'
    new_col_name = f'SUK{week + 1}'
    if col_name in SUK_Recorded_Total.columns:
        SUK_Recorded_Total[new_col_name] = SUK_Recorded_Total[col_name]

# Drop the original columns
SUK_Recorded_Total.drop(cols_to_select[:-1], axis=1, inplace=True)
# Select only the columns of float data type

float_cols = SUK_Recorded_Total.select_dtypes(include='float')

# Calculate the sum of hours for each row and add as a new column
SUK_Recorded_Total['SUK_Total_Hours'] = float_cols.iloc[:, :-1].sum(axis=1)




In [28]:
# Merging Dataframe
Weekly = Weekly.merge(SUK_Recorded_Total,on= 'Email', how ='left')

# SUK Recorded Hours for Weekly Average

# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    col_name = f'WK{week}_SUK_V'
    if col_name in data.columns:
        cols_to_select.append(col_name)

# Add the 'email' column to the list of selected columns
cols_to_select.append('Email')

# Select the desired columns and create a new DataFrame
SUK_Recorded_Average = data.loc[:, cols_to_select]

# Create new columns with 'SUK' prefix for each week
for week in range(end_week + 1):
    col_name = f'WK{week}_SUK_V'
    new_col_name = f'SUK{week+1}_Average'
    if col_name in SUK_Recorded_Average.columns:
        SUK_Recorded_Average[new_col_name] = SUK_Recorded_Average[col_name]

# Drop the original columns
SUK_Recorded_Average.drop(cols_to_select[:-1], axis=1, inplace=True)

# Select only the columns of float data type
float_cols = SUK_Recorded_Average.select_dtypes(include='float')

# Calculate the mean of the float columns
SUK_Recorded_Average['SUK_Average'] = float_cols.mean(axis=1)


# Merging Dataframe
Weekly = Weekly.merge(SUK_Recorded_Average,on= 'Email', how ='left')

# SUK Average Percentage

In [29]:
# Create a list of all column names up to the end week
cols_to_select = []
for week in range(end_week + 1):
    col_name = f'WK{week}_SUK_V'
    if col_name in df.columns:
        cols_to_select.append(col_name)

# Add the 'email' column to the list of selected columns
cols_to_select.append('Email')

# Select the desired columns and create a new DataFrame
SUK_Recorded_percent = df.loc[:, cols_to_select]

# Create new columns with 'SUK' prefix for each week
for week in range(end_week + 1):
    col_name = f'WK{week}_SUK_V'
    new_col_name = f'SUK{week+1}_percent'
    if col_name in SUK_Recorded_percent.columns:
        SUK_Recorded_percent[new_col_name] = SUK_Recorded_percent[col_name]

# Drop the original columns
SUK_Recorded_percent.drop(cols_to_select[:-1], axis=1, inplace=True)

# Select only the columns of float data type
float_cols = SUK_Recorded_percent.select_dtypes(include='float')

# Calculate the mean of the float columns
SUK_Recorded_percent['SUK_Percentage'] = float_cols.mean(axis=1)


In [30]:
# Merging Dataframe
Weekly = Weekly.merge(SUK_Recorded_percent,on= 'Email', how ='left')

# Weekly Masterclass total hours

In [31]:
# Create a list of all column names up to the end week
cols_to_select = [f'WK{i}_Master{j}' for i in range(end_week + 2) for j in range(1, 10) if f'WK{i}_Master{j}' in data.columns]

# Add the 'email' column to the list of selected columns
cols_to_select.append('Email')

# Select the desired columns and create a new DataFrame
Masterclass_Total = data.loc[:, cols_to_select]

# Rename columns as "Master{number}"
Masterclass_Total = Masterclass_Total.rename(columns={col_name: f'Master{i+1}' for i, col_name in enumerate(Masterclass_Total.columns) if col_name != 'Email'})

# Create the "Total_Hours" column
Masterclass_Total['Total_Hours'] = Masterclass_Total.select_dtypes(include=['float']).sum(axis=1)


In [32]:
# Merging Dataframe
Weekly = Weekly.merge(Masterclass_Total,on= 'Email', how ='left')

# Weekly Master Average

# Create a list of all column names up to the end week
cols_to_select = [f'WK{i}_Master{j}' for i in range(end_week + 2) for j in range(1, 10) if f'WK{i}_Master{j}' in data.columns]

# Add the 'email' column to the list of selected columns
cols_to_select.append('Email')

# Select the desired columns and create a new DataFrame
Masterclass_Avg = data.loc[:, cols_to_select]

# Rename columns as "Master{number}"
Masterclass_Avg = Masterclass_Avg.rename(columns={col_name: f'Master{i +1}_Average' for i, col_name in enumerate(Masterclass_Avg.columns) if col_name != 'Email'})

# Create the "Master_Average" column
Masterclass_Avg['Master_Average'] = Masterclass_Avg.select_dtypes(include=['float']).mean(axis=1)


# Merging Dataframe
Weekly = Weekly.merge(Masterclass_Avg,on= 'Email', how ='left')

# Masterclass Average Percent


In [33]:
# Create a list of all column names up to the end week
cols_to_select = [f'WK{i}_Master{j}' for i in range(end_week + 2) for j in range(1, 10) if f'WK{i}_Master{j}' in df.columns]

# Add the 'email' column to the list of selected columns
cols_to_select.append('Email')

# Select the desired columns and create a new DataFrame
Masterclass_Per = df.loc[:, cols_to_select]

# Rename columns as "Master{number}"
Masterclass_Per = Masterclass_Per.rename(columns={col_name: f'Master{i +1}_percent' for i, col_name in enumerate(Masterclass_Per.columns) if col_name != 'Email'})

# Create the "Master_Percentage" column
Masterclass_Per['Master_Percentage'] = Masterclass_Per.select_dtypes(include=['float']).mean(axis=1)


In [34]:
# Merging Dataframe
Weekly = Weekly.merge(Masterclass_Per,on= 'Email', how ='left')

In [35]:
Weekly.head(10)

,Email,WK0_Recorded_Total,WK1_Recorded_Total,WK2_Recorded_Total,WK3_Recorded_Total,WK4_Recorded_Total,WK5_Recorded_Total,WK6_Recorded_Total,WK7_Recorded_Total,WK8_Recorded_Total,All_Week_Total,Recorded_WK0_percent,Recorded_WK1_percent,Recorded_WK2_percent,Recorded_WK3_percent,Recorded_WK4_percent,Recorded_WK5_percent,Recorded_WK6_percent,Recorded_WK7_percent,Recorded_WK8_percent,Recorded_All_Week_percent,SUK1,SUK7,SUK_Total_Hours,SUK1_percent,SUK7_percent,SUK_Percentage,Master1,Master2,Master3,Master4,Master5,Total_Hours,Master1_percent,Master2_percent,Master3_percent,Master4_percent,Master5_percent,Master_Percentage
0,vijayacads@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.04,0.04,0.0,0.0,0.0,0.0,4.0,0.8
1,sreejith.sreenivasan@vigyanshaala.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.63,0.0,0.63,63.0,0.0,31.5,0.47,0.02,0.0,0.0,0.00,0.49,47.0,2.0,0.0,0.0,0.0,9.8
2,monikadaw5353@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,prachijagdale0123@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,nikitasingh4615@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,1.44,1.44,0.0,0.0,0.0,0.0,100.0,20.0
5,keerthishivampeta51@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
6,habibrutuja2003@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.13,0.00,0.0,0.0,0.00,0.13,13.0,0.0,0.0,0.0,0.0,2.6
7,florenceadekojo59@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
8,sreevishnumohanraj@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
9,bhargavigpatil64@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
Weekly.columns

Index(['Email', 'WK0_Recorded_Total', 'WK1_Recorded_Total',
       'WK2_Recorded_Total', 'WK3_Recorded_Total', 'WK4_Recorded_Total',
       'WK5_Recorded_Total', 'WK6_Recorded_Total', 'WK7_Recorded_Total',
       'WK8_Recorded_Total', 'All_Week_Total', 'Recorded_WK0_percent',
       'Recorded_WK1_percent', 'Recorded_WK2_percent', 'Recorded_WK3_percent',
       'Recorded_WK4_percent', 'Recorded_WK5_percent', 'Recorded_WK6_percent',
       'Recorded_WK7_percent', 'Recorded_WK8_percent',
       'Recorded_All_Week_percent', 'SUK1', 'SUK7', 'SUK_Total_Hours',
       'SUK1_percent', 'SUK7_percent', 'SUK_Percentage', 'Master1', 'Master2',
       'Master3', 'Master4', 'Master5', 'Total_Hours', 'Master1_percent',
       'Master2_percent', 'Master3_percent', 'Master4_percent',
       'Master5_percent', 'Master_Percentage'],
      dtype='object')

In [37]:
#Weekly.to_csv('Weekly_Attendence.csv', mode='a',index=False)


In [38]:
# Exporting data to Output files
Weekly.to_csv(r"C:\Users\HP\OneDrive - VigyanShaala\02 Products  Initiatives\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Remapping\Kalpana Output\Weekly_Attendence_Final_Sep_2023.csv",mode='w',index=False)

# Storing data to MySQL

In [39]:
# Connecting to MySQL Database
conn= msql.connect(host='localhost',user='root',password="VS@123",database="kalpana",auth_plugin='mysql_native_password')


In [40]:
cursor =conn.cursor()

In [41]:
# Get the existing columns in the database
cursor.execute("SHOW COLUMNS FROM 09_weekly_attendence")
existing_columns = [col[0] for col in cursor.fetchall()]

# Define the column name before which the new column should be added
target_column = 'Master_Percentage'

# Check if any new columns exist in the dataframe but not in the database
new_columns = [col for col in Weekly.columns if col not in existing_columns]
if new_columns:
    # Add new columns to the database before the target column
    for col in reversed(new_columns):
        if col not in existing_columns:
            # Get the index of the target column
            target_column_index = existing_columns.index(target_column)
            # Set the data type based on whether the column name starts with Comment
            data_type = "INT" 
            alter_query = f"ALTER TABLE 09_weekly_attendence ADD COLUMN {col} {data_type} AFTER {existing_columns[target_column_index]}"
            cursor.execute(alter_query)
            existing_columns.insert(target_column_index - 1, col)

In [42]:
# Your existing code for inserting data into the database table
for i, row in Weekly.iterrows():
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row] # replace "nan" values with None
    columns = ','.join(Weekly.columns)
    placeholders = ','.join(['%s']*len(row))
    # Construct the INSERT query with ON DUPLICATE KEY UPDATE clause
    query = f"INSERT INTO 09_weekly_attendence ({columns}) VALUES ({placeholders}) ON DUPLICATE KEY UPDATE "
    query += ", ".join([f"{col}=VALUES({col})" for col in Weekly.columns if col != 'Email'])
    # Execute the query
    cursor.execute(query, tuple(row))

In [43]:
conn.commit()